In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/MyDrive/Sun/ML Shock/Final project/
import sys
sys.path.insert(0,"C:/Users/Amy/Desktop/Green_Git/eegClassification/utils")

In [ ]:
from torch.utils.data import DataLoader
import pandas as pd
import os
import numpy as np

# import CustomDataset from CustomDataset.py from utils folder
from CustomDataset import CustomDataset

from scipy.special import kl_div

In [ ]:
# Load (train or test) data from csv file
path = "C:/Users/Amy/Desktop/Green_Git/eegClassification/sample_data/"
# path = "./data/"

data_type = "spec" # "eeg_raw" #"eeg_spec"  #
train = True
text = "train" if train else "test"

scaler_path = "C:/Users/Amy/Desktop/Green_Git/eegClassification/models/scalers/"
scaler_type = "min_max" #"standard" # 


In [ ]:

data_dir = (
    f"{text}_eegs/"
    if "eeg" in data_type
    else f"{text}_spectrograms/"
)
data_dir = path + data_dir

df = pd.read_csv(path + f"{text}.csv")

votes_cols = [
    "seizure_vote",
    "lpd_vote",
    "gpd_vote",
    "lrda_vote",
    "grda_vote",
    "other_vote",
]
label_cols = (
    ["eeg_id", "label_id", "eeg_label_offset_seconds"]
    if "eeg" in data_type
    else ["spectrogram_id", "label_id", "spectrogram_label_offset_seconds"]
)
offset = (
    ["eeg_label_offset_seconds"]
    if "eeg" in data_type
    else ["spectrogram_label_offset_seconds"]
)


In [ ]:
files = os.listdir(data_dir)
df = df[
    df["eeg_id" if "eeg" in data_type else "spectrogram_id"].isin(
        [int(f.split(".")[0]) for f in files]
    )
]

In [ ]:
# if info_cols not in df add it and set to zero
for col in offset:
    if col not in df.columns:
        df[col] = 0
# if df does not contain "label_id" add a unique label_id
if "label_id" not in df.columns:
    df["label_id"] = range(len(df))

info = {}
df_gr = df.groupby(label_cols)
for name, group in df_gr:
    # first row of group
    info[name] = {"votes": group[votes_cols].values[0] if train else None}


In [ ]:
# load scaler from path +f'{data_type}_standard_scaler.pkl' from pickle file
import pickle

with open(scaler_path + f"{data_type}_{scaler_type}_scaler.pkl", "rb") as f:
    scaler = pickle.load(f)


In [ ]:
dataset = CustomDataset(data_dir, data_type, info, transform=(lambda x: scaler.transform(x),))
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [72]:
def get_error(y_true, y_pred):
    return np.mean(kl_div(y_true, y_pred))


def evaluate(dataloader, y_pred):
    error = []
    count = 0
    for _, _, y in dataloader:
        for i in range(y.shape[0]):
            error.append(get_error(y[i].numpy(), y_pred[count]))
            count += 1
    return np.mean(error)

In [73]:
error = []
count = 0
dummy = [1/6 for _ in range(6)]
for _, _, y in dataloader:
    for i in range(y.shape[0]):
        error.append(get_error(y[i].numpy(), dummy))
        count += 1

print(f'error for uniform predictio: {sum(error)/count:.2f}')

error for uniform predictio: 0.19


## Gradient boost classifier

In [19]:
# # load the pca
# with open(path + f'{data_type}_pca.pkl', 'rb') as f:
#     pca = pickle.load(f)

In [74]:
# load the data
training_data = CustomDataset(data_dir, data_type = data_type)#, 
                            #   transform = (lambda x: standard_scaler.transform(x), lambda x: pca.transform(x.reshape(1,-1))))
train_dataloader = DataLoader(training_data, batch_size=16, shuffle=True)

In [75]:
# train a gradient boosting model to predict one of the 6 classes
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
for X, y, _ in train_dataloader:
    model.fit(X.reshape(X.shape[0], -1), y)



In [56]:
# compute the error on the training set
error = 0 
count = 0
for X, _, y in train_dataloader:
    y_pred = model.predict_proba(X.reshape(X.shape[0], -1))
    print(y.shape, y_pred.shape)
    for i in range(y.shape[0]):
        error += get_error(y[i].numpy(), y_pred[i])
        count += 1

print(f'error for the model: {error/count:.2f}')

torch.Size([14, 6]) (14, 2)


ValueError: operands could not be broadcast together with shapes (6,) (2,) 

In [48]:
# # predict the class probabilities
# y_train_pred = model.predict_proba(X_train_pca)
# y_valid_pred = model.predict_proba(X_valid_pca)
# y_test_pred = model.predict_proba(X_test_pca)

# # calculate the error
# error_train = [get_error(y_true, y_pred) for y_true, y_pred in zip(y_train, y_train_pred)]
# error_valid = [get_error(y_true, y_pred) for y_true, y_pred in zip(y_valid, y_valid_pred)]
# error_test = [get_error(y_true, y_pred) for y_true, y_pred in zip(y_test, y_test_pred)]

# print(f'train error: {np.mean(error_train):.2f}')
# print(f'valid error: {np.mean(error_valid):.2f}')
# print(f'test error: {np.mean(error_test):.2f}')